In [ ]:
!git clone https://github.com/MdAliAhnaf/frontal_3-category_face-mask_detection.git

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0

In [ ]:

!pip install tensorflow==2.10.1 numpy==1.26.4 keras==2.10
!pip install retina-face opencv-python pyyaml h5py
!pip install tensorflow-io
!pip install -U albumentations

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
import glob as gb
import scipy
import random
import seaborn as sns
import os
from PIL import Image
from tqdm import tqdm
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Ensure GPU Memory Growth
physical_devices = tf.config.experimental.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

print(physical_devices)

In [ ]:
resize = 224   #input image size
learning_rate = 1e-4
seed = 107
#define the hyperparamets for traing te neural network
INIT_LR = 1e-4
EPOCHS = 50
BS = 64
#dir_mask = 'drive/MyDrive/content/mask_dataset/Dataset/with_mask'
#dir_inc_mask = 'drive/MyDrive/content/mask_dataset/Dataset/mask_weared_incorrect'
#dir_nomask = 'drive/MyDrive/content/mask_dataset/Dataset/without_mask'
#TRAIN_DIR = 'drive/MyDrive/content/mask_dataset/Dataset'

dir_mask = '/kaggle/working/frontal_3-category_face-mask_detection/mask_dataset/with_mask'
dir_inc_mask = '/kaggle/working/frontal_3-category_face-mask_detection/mask_dataset/mask_weared_incorrect'
dir_nomask = '/kaggle/working/frontal_3-category_face-mask_detection/mask_dataset/without_mask'
TRAIN_DIR = '/kaggle/working/frontal_3-category_face-mask_detection/mask_dataset'

assert os.path.exists(dir_mask), 'Could not find' + dir_mask
assert os.path.exists(dir_inc_mask), 'Could not find' + dir_inc_mask
assert os.path.exists(dir_nomask), 'Could not find' + dir_nomask

In [ ]:
categories = []
class_count = []
train_exm = 0

for f in tqdm(os.listdir(TRAIN_DIR)):
    files = gb.glob(pathname=str(TRAIN_DIR  + '//' + f + '/*'))
    categories.append(f)
    class_count.append(len(files))
    train_exm += len(files)

sns.barplot(x=categories, y=class_count).set_title("distribution of train data")
plt.show()
print(train_exm)

# Joining code starts here
#CATEGORIES = ["with_mask", "mask_weared_incorrect", "without_mask"]
data = []
labels = []

for c in categories:
    path = os.path.join(TRAIN_DIR, c)
    for img in tqdm(os.listdir(path)):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(resize, resize))
        image = img_to_array(image)
        image = preprocess_input(image)
        
        data.append(image)
        labels.append(c)
        
data= np.array(data, dtype="float32")
labels = np.array(labels)

In [ ]:
len(labels)

In [ ]:
unique, counts = np.unique(labels, return_counts=True)
dict(zip(unique, counts))

In [ ]:
#Encode the labels in one hot encode form
lb = LabelEncoder()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
labels

In [ ]:
AugmentedData = ImageDataGenerator(
    zoom_range=0.15,  # Giới hạn phóng to/thu nhỏ từ 90% đến 110% của kích thước ảnh
    rotation_range=20,  # Giới hạn góc quay chỉ từ -15° đến 15°
    width_shift_range=0.2,  # Dịch chuyển chiều ngang tối đa 10%
    height_shift_range=0.2,  # Dịch chuyển chiều dọc tối đa 10%
    shear_range=0.15,  # Giới hạn độ biến dạng shear
    horizontal_flip=True,  # Lật ảnh ngang
    fill_mode="nearest"  # Sử dụng giá trị gần nhất để điền vào các vùng trống sau khi biến đổi
)

In [ ]:
#define the model
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
baseModel.summary()

# construct the head of the model that will be placed on top of the
# the base model (A simple CNN as the Head model) 
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.25)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
#divide data into training and testing sets
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.2, stratify=labels, random_state=42)
print(f"Shape of x_train: {trainX.shape}")
print(f"Shape of y_train: {trainY.shape}")
print()
print(f"Shape of x_test: {testX.shape}")
print(f"Shape of y_test: {testY.shape}")

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Define early stopping and model checkpoint callbacks
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("best_model.h5", monitor="val_loss", save_best_only=True, verbose=1)

# Compile the model and train it
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

print("""[INFO] compiling model...
[INFO] training head...""")

# Train the model with the callbacks
H = model.fit(
	AugmentedData.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS,
	callbacks=[early_stopping, model_checkpoint]
)

In [ ]:
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs
	))

# # serialize the model to disk
# print("[INFO] saving mask detector model...")

# plot the training loss and accuracy
# Sửa lại đoạn mã vẽ
N = len(H.history["loss"])  # Lấy số epoch thực tế từ quá trình huấn luyện
plt.style.use("ggplot")
plt.figure()

# Vẽ đồ thị
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")

# Thêm các phần tử đồ thị khác
plt.title("Training Loss and Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()


In [ ]:
model.save("my_mask_detector_1.h5")